In [1]:
import pyvista
import openep
from openep._datasets.openep_datasets import DATASET_2

In [2]:
pyvista.set_jupyter_backend('ipygany')

In [3]:
case = openep.load_openep_mat(DATASET_2)
mesh = case.create_mesh()

In [4]:
mesh.plot()

Scene(background_color='#4c4c4c', camera={'position': [71.80763645292058, -4.5938635470794225, 216.89163645292…

In [5]:
plotter = openep.draw.draw_map(mesh, field=case.fields.bipolar_voltage, add_mesh_kws={'cmap': 'Plasma'})
plotter.show()


/usr/local/Caskroom/miniconda/base/envs/openep/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


AppLayout(children=(VBox(children=(HTML(value='<h3></h3>'), Dropdown(description='Colormap:', options={'BrBG':…